In [43]:
import fastf1 as ff1
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
ff1.Cache.enable_cache('cache')
import numpy as np


In [77]:
session = ff1.get_session(2023, 'Suzuka', 'FP3')
session.load()

core           INFO 	Loading data for Japanese Grand Prix - Practice 3 [v3.1.2]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '40', '44', '55', '63', '77', '81']


In [78]:
session.laps.base_class_view.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate'],
      dtype='object')

In [79]:
df = session.laps.base_class_view[['Driver','Team', 'LapTime', 'LapNumber', 'Stint','Compound', 'TyreLife']]
df['LapTime_seconds'] = df['LapTime'].dt.total_seconds()
avg_times = df.groupby('Driver')['LapTime_seconds'].mean().sort_values()

df['stint_compound']= df['Compound'] + '_' +df['Stint'].astype('str')
df = df.sort_values(by=['Stint'])
colors = {}
for sint_compound in df['stint_compound'].unique():
    if 'HARD' in sint_compound:
        colors[sint_compound] = 'black'
    if 'MEDIUM' in sint_compound:
        colors[sint_compound] = 'yellow'
    if 'SOFT' in sint_compound:
        colors[sint_compound] = 'red'


In [81]:
import plotly.express as px

# Création du boxplot avec plotly
fig = px.box(
    df,
    x="Driver",
    y="LapTime_seconds",
    color="stint_compound",
    category_orders={"Driver": avg_times.index.tolist()},  # pour ordonner les pilotes
    color_discrete_map=colors,  # utilisez votre dictionnaire de couleurs
    title='Boxplot of Lap Times by Driver and Stint',
    labels={"LapTime_seconds": "Lap Time (seconds)", "Driver": "Driver"},
    points='all',    
    )
# Rotation des labels de l'axe x
fig.update_layout(xaxis_tickangle=-45)

#yaxis_range=[df['LapTime_seconds'].min(), 105]
# Affichage du graphique
fig.show()
